In [ ]:
import os
import trimesh
import pyrender
import numpy as np
from get_mesh import as_mesh
import matplotlib.pyplot as plt
import imageio

In [ ]:
egypt = trimesh.load('../Dataset/egypt_model/baked_mesh.obj')
egypt_trimesh = as_mesh(egypt)
egypt_mesh = pyrender.Mesh.from_trimesh(egypt_trimesh)

picture_folder = 'egypt_picture_data'
os.makedirs(picture_folder, exist_ok=True)

In [ ]:
def generate_points(radius, num_points):
    theta_values = np.linspace(0, np.pi, num_points)
    phi_values = np.linspace(0, np.pi, num_points)
    points = []

    for phi in phi_values:
        for theta in theta_values:
            x = radius * np.cos(theta) * np.cos(phi)
            y = radius * np.sin(theta) * np.cos(phi)
            z = radius * np.sin(phi)
            points.append((x, z, y))

    return points



focal = 0.8

points_on_semicircle = generate_points(focal, 10)

In [ ]:
camera_positions = []

for i, point in enumerate(points_on_semicircle):
    camera_position =  np.array(point)
    camera_target = np.array([0,0,0])
    # D = camera_target - camera_position        # camera direction
    D = camera_position - camera_target
    D = D / np.linalg.norm(D) if np.linalg.norm(D) != 0 else D
    # print(f'D {D}')

    up = np.array([0,1,0])
    R = np.cross(up, D)
    R = R / np.linalg.norm(R) if np.linalg.norm(R) != 0 else R
    # print(f'R {R}')

    U = np.cross(D, R)
    # print(f'U {U}')

    rotation = np.eye(4)
    rotation[0:3, 0] = R
    rotation[0:3, 1] = U
    rotation[0:3, 2] = D
    # print(rotation)

    import copy
    r_t = copy.deepcopy(rotation)
    r_t[0:3,3] = camera_position
    camera_positions.append(r_t)

np.savez('camera_positions.npz', arr1=camera_positions)

In [ ]:
scene = pyrender.Scene()
scene.add(egypt_mesh, name='egypt')

for i, camera_pose in enumerate(camera_positions):
    camera = pyrender.PerspectiveCamera(yfov=np.pi / 3.0, aspectRatio=1.0)
    camera_node = scene.add(camera, pose=camera_pose)

    DireLight = pyrender.DirectionalLight(color=np.ones(3), intensity=15.0)
    light_node = scene.add(DireLight, pose=camera_pose)

    r = pyrender.OffscreenRenderer(viewport_height=1800, viewport_width=1800)
    color, depth = r.render(scene)

    img_filename = os.path.join(picture_folder, f'egypt_image_{i}.png')
    imageio.imwrite(img_filename, color)

    scene.remove_node(camera_node)
    scene.remove_node(light_node)
    r.delete()